# Create dataset from given noisy data


# 1)- Import Key Modules

In [1]:
# support both Python 2 and Python 3 with minimal overhead.
from __future__ import absolute_import, division, print_function

# I am an engineer. I care only about error not warning. So, let's be maverick and ignore warnings.
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pickle
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re
from bs4 import BeautifulSoup
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

# 2)- Loading Data

In [3]:
data=pd.read_pickle('file_clean_intent.pkl')
data.shape

(1000, 4)

In [4]:
data.head()

,firstmessage,dep,clean,clean2
0,Hello:<br><br>I tried to apply a voucher to th...,Shipping issues,Hello br br I tried to apply a voucher to this...,hello: tried apply voucher order received mail...
1,"<br>Hi,<br><br>Do you have a tracking number p...",Shipping issues,br Hi br br Do you have a tracking number plea...,"hi, tracking number please? thank you. kind re..."
2,"<p>Hi,<br /><br />I&rsquo;ve received my order...",product complaints - products (Reklamation Pro...,p Hi br br I rsquo ve received my order but on...,"hi, i&rsquo;ve received order one poster bad s..."
3,I was asked to test a saal photobook and I was...,ShareWithSaal,I wa asked to test a saal photobook and I wa s...,asked test saal photobook delighted result! ar...
4,<p><br />Ainda n&atilde;o recebi a encomenda e...,Order management,p br Ainda n atilde o recebi a encomenda e est...,ainda n&atilde;o recebi encomenda e estive sem...


In [5]:
def print_plot(index):
    example = data[data.index == index][['firstmessage', 'dep']].values[0]
    if len(example) > 0:
        print(example[0])
        print('Label:', example[1])
print_plot(10)

Amazing website, company and product! As a professional photographer through 5 years, I can vouch for Saal Digital and their high quality service
Label: Customer feedback


In [6]:
print_plot(2)

<p>Hi,<br /><br />I&rsquo;ve received my order but one of the posters was in bad state.&nbsp;<br /><br />It was folded. &nbsp;La post sent me an email saying they had to &ldquo;reinforce&rdquo; the packaging.<br /><br />The order was 512005251803340<br /><br />Regards,<br /><br />Eugenia Sebastiani&nbsp;<br /><br /></p>
Label: product complaints - products (Reklamation Produkte)


In [7]:
data['firstmessage'].apply(lambda x: len(x.split(' '))).sum()

105422

### Triming

In [8]:
print_plot(30)

<br>Hello,<br><br>I wanted to see if I could change my delivery address for order 512005290316262.<br><br>I'd like to change it to:<br>200 Robert Rd Dedham MA 02026 United States<br><br>Thanks! <br><br>On Thu, May 28, 2020 at 9:22 PM <mailto:support@saal-digital.com> wrote:<br><br>    |     http://www.saal-digital.com/<br>    |<br>Dear Mr. Peters,  thank you for your order. We are very pleased that you have chosen  www.saal-digital.com.<br>Your order number:  https://www.saal-digital.com/ordercockpit/?email=mikemppeters%40gmail.com&ordernumber=512005290316262<br>    |   Your order<br>    |   Article    |   Quantity    |   Total price<br>    |   8 x 8 Photobook Professional Line, Spreads: matte photo paper, 32 pages     |    1    |    $ 43,29<br>    |   Premium gift box black     |    1    |    $ 60,00<br>    |   Cover: Leather / Linen, Cover surface: leather black, Barcode: without barcode     |    1    |    $ 52,50<br>    |   Shipping costs:    |   $ 4,95<br>    |   Voucher    |   - $

In [9]:
print_plot(28)

<br>Witam, w przesyłce brakuje puzzli.<br><br><br>There's no puzzle in the parcel.<br><br><br>Pozdrawiam,<br><br>Barbara Fuchs<br><br>pt., 29 maj 2020, 16:30 użytkownik <mailto:support@saal-digital.pl> napisał:<br><br>    |    http://www.saal-digital.pl/<br>    |<br>Szanowna Pani Fuchs,  W załączniku otrzymają Państwo fakturę.<br>Obsługa klienta Są Państwo zadowoleni z naszego oprogramowania / naszego sklepu internetowego? Będziemy wdzięczni za  Państwa opinię. Każda opinia pomaga nam w dalszym doskonaleniu i realizowaniu życzeń klientów. Będziemy wdzięczni, jeśli zdecydują się Państwo w przyszłości na Saal Digital. Tymczasem życzymy pozytywnych wrażeń z Państwa zamówieniem.  Wskazówka:  Państwa adresu e-mail użyjemy przy opracowywaniu zamówienia oraz po to, aby poinformować Państwo e-mailem o naszych towarach i usługach. W każdej chwili mogą Państwo sprzeciwić się użyciu swojego adresu e-mail. Jeśli nie życzą sobie Państwo otrzymywać dalszych reklam, proszę kliknąć na link  Usuń profi

In [10]:
print_plot(10)

Amazing website, company and product! As a professional photographer through 5 years, I can vouch for Saal Digital and their high quality service
Label: Customer feedback


In [11]:
print_plot(120)

<b>Merged from ticket PLSD200006295</b><br><br>Witam,<br> <br>Złożyłem zamówienie na foto książkę 29.05.2020 , płatność została zrealizowana ,<br> <br>Komunikat potwierdzający złożenie zamówienia został potwierdzony informacją – płatność za zamówienie 502005290818113 zakończona powodzeniem – szczegóły poniżej , natomiast nie dostałem informacji kiedy zamówienie zostanie zrealizowane.<br> <br>Bardzo proszę o informację kiedy foto książka zostanie dostarczona ?<br> <br>Z uszanowaniem<br>Jakub Mołdawa<br><br><br><br>    |   http://www.avg.com/email-signature?utm_medium=email&utm_source=link&utm_campaign=sig-email&utm_content=emailclient    |    Wolny od wirusów.  www.avg.com     #DAB4FAD8-2DD7-40BB-A1B8-4E2AA1F9FDF2
Label: Order management


In [12]:
print_plot(24)

<br>  <br> From:  Eva Herčková <evaherec11@gmail.com>  Sent: Mittwoch, 3. Juni 2020 11:23 To: Florian Stellwag <florian.stellwag@saal-digital.de> Subject: Re: Důležité informace pro distributory<br> <br><br>Dobrý den, bohužel mi odkaz nefunguje. Měla bych zájem, právě teď mám zakázku na knihu Professional. Sleva pro distributory mi skončila v roce 2019, nyní vždy čerpám když máte jakékoliv akce. Tato sleva nelze již prodloužit?<br><br>Děkuji<br><br>Eva Herčková<br> <br><br><br>st 3. 6. 2020 v 10:05 odesílatel Florian Stellwag <mailto:florian.stellwag@saal-digital.de> napsal:<br><br>    |<br>http://www.saal-digital.cz/<br> <br>    |<br>    |<br><br>Vážená paní Hercková,<br>  <br>    |<br><br>v časech jako jsou tyto bychom měli držet spolu, proto Vám prodloužíme<br>Vaši slevu pro distributory do 31.01.2021, bez ohledu na prodej v roce 2020.<br>  <br>    |<br><br>Tímto způsobem Vám chceme nabídnout veškerou naši podporu a co nejjednodušší návrat k fotografické činnosti. Chcete-li využít s

In [13]:
text_remove=data.firstmessage[30]
text_remove

"<br>Hello,<br><br>I wanted to see if I could change my delivery address for order\xa0512005290316262.<br><br>I'd like to change it to:<br>200 Robert Rd Dedham MA 02026 United States<br><br>Thanks!\xa0<br><br>On Thu, May 28, 2020 at 9:22 PM <mailto:support@saal-digital.com> wrote:<br><br>    |     http://www.saal-digital.com/<br>    |<br>Dear Mr. Peters,  thank you for your order. We are very pleased that you have chosen  www.saal-digital.com.<br>Your order number:  https://www.saal-digital.com/ordercockpit/?email=mikemppeters%40gmail.com&ordernumber=512005290316262<br>    |   Your order<br>    |   Article    |   Quantity    |   Total price<br>    |   8\xa0x\xa08 Photobook Professional Line, Spreads: matte photo paper, 32 pages     |    1    |    $ 43,29<br>    |   Premium gift box black     |    1    |    $ 60,00<br>    |   Cover: Leather\xa0/\xa0Linen, Cover surface: leather black, Barcode: without barcode     |    1    |    $ 52,50<br>    |   Shipping costs:    |   $ 4,95<br>    |  

In [14]:
type(text_remove)

str

In [15]:
text_remove.split("support@saal-digital.com",maxsplit=1)[0]

"<br>Hello,<br><br>I wanted to see if I could change my delivery address for order\xa0512005290316262.<br><br>I'd like to change it to:<br>200 Robert Rd Dedham MA 02026 United States<br><br>Thanks!\xa0<br><br>On Thu, May 28, 2020 at 9:22 PM <mailto:"

In [16]:
data.head()

,firstmessage,dep,clean,clean2
0,Hello:<br><br>I tried to apply a voucher to th...,Shipping issues,Hello br br I tried to apply a voucher to this...,hello: tried apply voucher order received mail...
1,"<br>Hi,<br><br>Do you have a tracking number p...",Shipping issues,br Hi br br Do you have a tracking number plea...,"hi, tracking number please? thank you. kind re..."
2,"<p>Hi,<br /><br />I&rsquo;ve received my order...",product complaints - products (Reklamation Pro...,p Hi br br I rsquo ve received my order but on...,"hi, i&rsquo;ve received order one poster bad s..."
3,I was asked to test a saal photobook and I was...,ShareWithSaal,I wa asked to test a saal photobook and I wa s...,asked test saal photobook delighted result! ar...
4,<p><br />Ainda n&atilde;o recebi a encomenda e...,Order management,p br Ainda n atilde o recebi a encomenda e est...,ainda n&atilde;o recebi encomenda e estive sem...


In [19]:
#df['text_new'] = df['text'].str.split('::').str[0]

In [20]:
data['trim']=data['firstmessage'].str.split("support@saal-digital.com").str[0]

In [21]:
data.trim[0]

"Hello:<br><br>I tried to apply a voucher to this order that I received in the mail but it didn't appear on the order total:<br><br> Your\xa0$40 \xa0voucher from $129.99:\xa0 RFK93R3AnF-PVG\xa0<br><br>Can you please apply this to my order?<br><br>Thank you,<br>Colin\xa0<br><br>On Sat, Jun 6, 2020 at 2:49 PM <mailto:"

In [22]:
data.firstmessage[0]

"Hello:<br><br>I tried to apply a voucher to this order that I received in the mail but it didn't appear on the order total:<br><br> Your\xa0$40 \xa0voucher from $129.99:\xa0 RFK93R3AnF-PVG\xa0<br><br>Can you please apply this to my order?<br><br>Thank you,<br>Colin\xa0<br><br>On Sat, Jun 6, 2020 at 2:49 PM <mailto:support@saal-digital.com> wrote:<br><br>    |     http://www.saal-digital.com/<br>    |<br>Dear Mr. Shreffler,  thank you for your order. We are very pleased that you have chosen  www.saal-digital.com.<br>Your order number:  https://www.saal-digital.com/ordercockpit/?email=c%40colinshreffler.com&ordernumber=802006062223358<br>    |   Your order<br>    |   Article    |   Quantity    |   Total price<br>    |   Metal Print 19.7\xa0x\xa029.5, Metal Print, aluminium subframe     |    1    |    $ 85,39<br>    |   Metal Print 19.7\xa0x\xa029.5, Metal Print, aluminium subframe     |    1    |    $ 85,39<br>    |   Metal Print 11.8\xa0x\xa017.7, Metal Print, aluminium subframe     | 

In [23]:
data.trim[30]

"<br>Hello,<br><br>I wanted to see if I could change my delivery address for order\xa0512005290316262.<br><br>I'd like to change it to:<br>200 Robert Rd Dedham MA 02026 United States<br><br>Thanks!\xa0<br><br>On Thu, May 28, 2020 at 9:22 PM <mailto:"

In [24]:
data.firstmessage[30]

"<br>Hello,<br><br>I wanted to see if I could change my delivery address for order\xa0512005290316262.<br><br>I'd like to change it to:<br>200 Robert Rd Dedham MA 02026 United States<br><br>Thanks!\xa0<br><br>On Thu, May 28, 2020 at 9:22 PM <mailto:support@saal-digital.com> wrote:<br><br>    |     http://www.saal-digital.com/<br>    |<br>Dear Mr. Peters,  thank you for your order. We are very pleased that you have chosen  www.saal-digital.com.<br>Your order number:  https://www.saal-digital.com/ordercockpit/?email=mikemppeters%40gmail.com&ordernumber=512005290316262<br>    |   Your order<br>    |   Article    |   Quantity    |   Total price<br>    |   8\xa0x\xa08 Photobook Professional Line, Spreads: matte photo paper, 32 pages     |    1    |    $ 43,29<br>    |   Premium gift box black     |    1    |    $ 60,00<br>    |   Cover: Leather\xa0/\xa0Linen, Cover surface: leather black, Barcode: without barcode     |    1    |    $ 52,50<br>    |   Shipping costs:    |   $ 4,95<br>    |  

In [25]:
data.trim[24]

'<br> \xa0<br> From:  Eva Herčková <evaherec11@gmail.com>  Sent: Mittwoch, 3. Juni 2020 11:23 To: Florian Stellwag <florian.stellwag@saal-digital.de> Subject: Re: Důležité informace pro distributory<br>\xa0<br><br>Dobrý den, bohužel mi odkaz nefunguje. Měla bych zájem, právě teď mám zakázku na knihu Professional. Sleva pro distributory mi skončila v roce 2019, nyní vždy čerpám když máte jakékoliv akce. Tato sleva nelze již prodloužit?<br><br>Děkuji<br><br>Eva Herčková<br>\xa0<br><br><br>st 3. 6. 2020 v\xa010:05 odesílatel Florian Stellwag <mailto:florian.stellwag@saal-digital.de> napsal:<br><br>    |<br>http://www.saal-digital.cz/<br>\xa0<br>    |<br>    |<br><br>Vážená paní Hercková,<br> \xa0<br>    |<br><br>v časech jako jsou tyto bychom měli držet spolu, proto Vám prodloužíme<br>Vaši slevu pro distributory do 31.01.2021, bez ohledu na prodej v roce 2020.<br> \xa0<br>    |<br><br>Tímto způsobem Vám chceme nabídnout veškerou naši podporu a co nejjednodušší návrat k fotografické činnos

In [26]:
data.firstmessage[24]

'<br> \xa0<br> From:  Eva Herčková <evaherec11@gmail.com>  Sent: Mittwoch, 3. Juni 2020 11:23 To: Florian Stellwag <florian.stellwag@saal-digital.de> Subject: Re: Důležité informace pro distributory<br>\xa0<br><br>Dobrý den, bohužel mi odkaz nefunguje. Měla bych zájem, právě teď mám zakázku na knihu Professional. Sleva pro distributory mi skončila v roce 2019, nyní vždy čerpám když máte jakékoliv akce. Tato sleva nelze již prodloužit?<br><br>Děkuji<br><br>Eva Herčková<br>\xa0<br><br><br>st 3. 6. 2020 v\xa010:05 odesílatel Florian Stellwag <mailto:florian.stellwag@saal-digital.de> napsal:<br><br>    |<br>http://www.saal-digital.cz/<br>\xa0<br>    |<br>    |<br><br>Vážená paní Hercková,<br> \xa0<br>    |<br><br>v časech jako jsou tyto bychom měli držet spolu, proto Vám prodloužíme<br>Vaši slevu pro distributory do 31.01.2021, bez ohledu na prodej v roce 2020.<br> \xa0<br>    |<br><br>Tímto způsobem Vám chceme nabídnout veškerou naši podporu a co nejjednodušší návrat k fotografické činnos

# END OF NOTEBOOK